In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import collections
import math
import sys
import numpy as np
# sys.path.append("/home/rishi/Projects/Pyramid-fuse/")
# # sys.setrecursionlimit(10000000)
# import Utils
# # from Utils.CubePad import CustomPad
image_path = 'Sample.jpg'
# import torchvision.models as models
from pspnet import PSPNet

ModuleNotFoundError: No module named 'utils'

In [34]:
import numpy as np

In [35]:
from PIL import Image
import cv2

In [38]:
img = np.asarray(Image.open(image_path), dtype=np.float32)
img=img.transpose(2,0,1)
img=np.expand_dims(img,0)
img=torch.tensor(img)

In [5]:
class CETransform(nn.Module):
    def __init__(self):
        super(CETransform, self).__init__()
        equ_h = [512, 128, 64, 32, 16]
        cube_h = [256, 64, 32, 16, 8]

        self.c2e = dict()
        self.e2c = dict()

        for h in equ_h:
            a = Utils.Equirec2Cube(1, h, h*2, h//2, 90)
            self.e2c['(%d,%d)' % (h, h*2)] = a

        for h in cube_h:
            a = Utils.Cube2Equirec(1, h, h*2, h*4)
            self.c2e['(%d)' % (h)] = a

    def E2C(self, x):
        print(x.shape)
        [bs, c, h, w] = x.shape
        key = '(%d,%d)' % (h, w)
        print(key)
        assert key in self.e2c
        return self.e2c[key].ToCubeTensor(x)

    def C2E(self, x):
        print(x.shape)
        [bs, c, h, w] = x.shape
        key = '(%d)' % (h)
        assert key in self.c2e and h == w
        return self.c2e[key].ToEquirecTensor(x)

    def forward(self, equi, cube):
        return self.e2c(equi), self.c2e(cube)


In [6]:
mypad = getattr(Utils.CubePad, 'SpherePad')

In [7]:
ce = CETransform()

In [8]:
img1 = torch.rand((1,3,512,1024)).cuda()

In [9]:
img1.shape

torch.Size([1, 3, 512, 1024])

In [10]:
test = ce.E2C(img1)

torch.Size([1, 3, 512, 1024])
(512,1024)
tensor([[[[0.5428, 0.3148, 0.6410,  ..., 0.6896, 0.7269, 0.0994],
          [0.3571, 0.4063, 0.9146,  ..., 0.7948, 0.0436, 0.4228],
          [0.2656, 0.1951, 0.4688,  ..., 0.5246, 0.2480, 0.3659],
          ...,
          [0.2175, 0.3050, 0.5544,  ..., 0.6198, 0.9992, 0.6829],
          [0.2661, 0.6307, 0.5076,  ..., 0.5635, 0.9980, 0.1390],
          [0.6264, 0.3644, 0.1565,  ..., 0.4353, 0.5788, 0.2921]],

         [[0.7930, 0.1298, 0.1656,  ..., 0.3624, 0.8011, 0.4380],
          [0.7909, 0.7882, 0.3560,  ..., 0.6038, 0.5465, 0.2286],
          [0.8887, 0.0173, 0.8868,  ..., 0.5212, 0.7768, 0.6381],
          ...,
          [0.3020, 0.5898, 0.7752,  ..., 0.1172, 0.4321, 0.9730],
          [0.5912, 0.8841, 0.1362,  ..., 0.6778, 0.1112, 0.3570],
          [0.3691, 0.5098, 0.1221,  ..., 0.4269, 0.7718, 0.6942]],

         [[0.4914, 0.5273, 0.4225,  ..., 0.0191, 0.7941, 0.3177],
          [0.8151, 0.6494, 0.4731,  ..., 0.4635, 0.6747, 0.4139],
 

/home/rishi/.local/lib/python3.6/site-packages/torch/nn/functional.py:3226: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


In [11]:
test2 = ce.C2E(test)

torch.Size([6, 3, 256, 256])


In [7]:
res = models.resnet50(pretrained=True)

In [39]:
out = res(img)

In [40]:
out.shape

torch.Size([1, 1000])

In [3]:
torch.empty(4)

tensor([-1.5645e-24,  3.0872e-41, -1.5645e-24,  3.0872e-41])